Test du projet de class trajets indirect

In [2]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import traitement_od as t
import pandas as pd
import numpy as np
import altair as alt
import datetime as dt
pd.set_option('display.max_colwidth', 1000)
import os
from sklearn.cluster import DBSCAN

In [3]:
%time df_3semaines=t.ouvrir_fichier_lapi('2019-01-29 00:00:00','2019-02-17 23:59:59').set_index('created').sort_index()

Wall time: 1min 30s


In [12]:
df_journee=t.ouvrir_fichier_lapi('2019-01-29','2019-01-31')

In [133]:
df_journee

,camera_id,immat,fiability,l,state
created,,,,,
2019-01-29 00:00:02,13,8b1b888305c26503d9b83b38ea6516402d985c5f,99,-1,BE
2019-01-29 00:00:03,8,c0dd4662304e0901a0866dabc8de414d8e19206d,99,0,IT
2019-01-29 00:00:03,14,d17d965f7b65bfb2fd814867bd3c714968d07328,99,-1,PT
2019-01-29 00:00:04,5,9713b42132193b69c5a68909f0ee2630a06bc8e0,99,0,FR
2019-01-29 00:00:05,10,1786fdedee0acfb8629e9b5cd61104d77bce066f,39,0,ES
2019-01-29 00:00:05,3,b9faa07d52299826c97a48b9974cdf7e17bf55d8,99,0,FR
2019-01-29 00:00:06,3,332a381c558c51acef76ab04873f438bd530015d,99,0,FR
2019-01-29 00:00:10,3,a099baf2aac15755c014ba2593797dcd903b95fb,89,0,FR
2019-01-29 00:00:13,3,13c9b885389ad4d59feaaac9eec0b330349c33f4,99,0,IT


In [745]:
%time df_3semaines.loc[pd.to_datetime('2019-01-29'):pd.to_datetime('2019-01-30')]

Wall time: 0 ns


,camera_id,immat,fiability,l,state
created,,,,,
2019-01-29 00:00:02,13,8b1b888305c26503d9b83b38ea6516402d985c5f,99,-1,BE
2019-01-29 00:00:03,8,c0dd4662304e0901a0866dabc8de414d8e19206d,99,0,IT
2019-01-29 00:00:03,14,d17d965f7b65bfb2fd814867bd3c714968d07328,99,-1,PT
2019-01-29 00:00:04,5,9713b42132193b69c5a68909f0ee2630a06bc8e0,99,0,FR
2019-01-29 00:00:05,10,1786fdedee0acfb8629e9b5cd61104d77bce066f,39,0,ES
2019-01-29 00:00:05,3,b9faa07d52299826c97a48b9974cdf7e17bf55d8,99,0,FR
2019-01-29 00:00:06,3,332a381c558c51acef76ab04873f438bd530015d,99,0,FR
2019-01-29 00:00:10,3,a099baf2aac15755c014ba2593797dcd903b95fb,89,0,FR
2019-01-29 00:00:13,3,13c9b885389ad4d59feaaac9eec0b330349c33f4,99,0,IT


LISTE DES TRAJETS

In [153]:
liste_trajet_locale=(pd.DataFrame([{'origine':'A63','destination':'A10','cam_o':14, 'cam_d':11, 'trajets':[
                                                        {'cameras':[14,19,4,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,1,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,4,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,1,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,4,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,1,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,4,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,1,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,11],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A63','cam_o':12, 'cam_d':13,'trajets':[{'cameras':[12,6,2,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,2,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,2,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,2,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A63','destination':'N10','cam_o':14, 'cam_d':5,'trajets':[{'cameras':[14,19,4,5],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,1,5],'type_trajet':'indirect'},
                                                        {'cameras':[14,4,5],'type_trajet':'indirect'},
                                                        {'cameras':[14,1,5],'type_trajet':'indirect'},
                                                        {'cameras':[14,5],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'N10','destination':'A63','cam_o':6, 'cam_d':13,'trajets':[{'cameras':[6,2,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,2,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A62','destination':'A10','cam_o':10, 'cam_d':11,'trajets':[{'cameras':[10,4,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[10,4,11],'type_trajet':'indirect'},
                                                        {'cameras':[10,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[10,11],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A62','cam_o':12, 'cam_d':9,'trajets':[{'cameras':[12,6,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[12,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,9],'type_trajet':'indirect'},
                                                        {'cameras':[12,9],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A62','destination':'N10','cam_o':10, 'cam_d':5,'trajets':[{'cameras':[10,4,5],'type_trajet':'indirect'},
                                                        {'cameras':[10,5],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'N10','destination':'A62','cam_o':6, 'cam_d':9,'trajets':[{'cameras':[6,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[6,9],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A63','destination':'A62','cam_o':14, 'cam_d':9,'trajets':[{'cameras':[14,19,9],'type_trajet':'indirect'},
                                                        {'cameras':[14,9],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A62','destination':'A63','cam_o':10, 'cam_d':13,'trajets':[{'cameras':[10,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[10,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A89','destination':'A63','cam_o':8 ,'cam_d':13,'trajets':[{'cameras':[8,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[8,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[8,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[8,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A63','destination':'A89','cam_o':14, 'cam_d':7,'trajets':[{'cameras':[14,19,4,7],'type_trajet':'indirect'},
                                                        {'cameras':[14,4,7],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,7],'type_trajet':'indirect'},
                                                        {'cameras':[14,7],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A89','destination':'A62','cam_o':8, 'cam_d':9,'trajets':[{'cameras':[8,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[8,9],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A62','destination':'A89','cam_o':10, 'cam_d':7,'trajets':[{'cameras':[10,4,7],'type_trajet':'indirect'},
                                                        {'cameras':[10,7],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A89','destination':'A10','cam_o':8, 'cam_d':11,'trajets':[{'cameras':[8,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[8,11],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A89','cam_o':12, 'cam_d':7,'trajets':[{'cameras':[12,6,7],'type_trajet':'indirect'},
                                                        {'cameras':[12,7],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A89','destination':'N10','cam_o':8, 'cam_d':5,'trajets':[{'cameras':[8,5],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'N10','destination':'A89','cam_o':6, 'cam_d':7,'trajets':[{'cameras':[6,7],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A630','cam_o':12, 'cam_d':18,'trajets':[{'cameras':[12,6,2,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,6,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,2,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,6,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,18],'type_trajet':'direct'},
                                                        ]},
                            {'origine':'A630','destination':'A10','cam_o':19, 'cam_d':18,'trajets':[{'cameras':[19,4,5,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,1,5,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,1,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,4,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,5,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,11],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A630','destination':'A62','cam_o':19, 'cam_d':9,'trajets':[{'cameras':[19,9],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'A62','destination':'A630','cam_o':10, 'cam_d':18,'trajets':[{'cameras':[10,18],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'A630','destination':'A89','cam_o':19, 'cam_d':7,'trajets':[{'cameras':[19,4,7],'type_trajet':'indirect'},
                                                         {'cameras':[19,7],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'A89','destination':'A630','cam_o':8, 'cam_d':18,'trajets':[{'cameras':[8,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[8,18],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'N10','destination':'A630','cam_o':6, 'cam_d':18,'trajets':[{'cameras':[6,2,18],'type_trajet':'indirect'},
                                                         {'cameras':[6,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[6,18],'type_trajet':'direct'},
                                                        ]},
                            {'origine':'A630','destination':'N10','cam_o':19, 'cam_d':5,'trajets':[{'cameras':[19,1,5],'type_trajet':'indirect'},
                                                         {'cameras':[19,4,5],'type_trajet':'indirect'},
                                                         {'cameras':[19,5],'type_trajet':'direct'},
                                                        ]}
                           ]))[['origine', 'destination', 'cam_o', 'cam_d','trajets']]

In [730]:
%time test=t.trajet(df_journee, '2019-01-29 08:00:00', 60, 16, [6,13], type='Global')

Wall time: 6.33 s


In [14]:
test=t.transit_1_jour(df_journee,'2019-01-29 06:00:00', liste_trajet_locale)

date : 2019-01-29 06:00:00 debut_traitement : 2019-04-04 11:37:56.280046
date : 2019-01-29 07:00:00 debut_traitement : 2019-04-04 11:38:30.202866
date : 2019-01-29 08:00:00 debut_traitement : 2019-04-04 11:39:24.455723
date : 2019-01-29 09:00:00 debut_traitement : 2019-04-04 11:40:30.992822
date : 2019-01-29 10:00:00 debut_traitement : 2019-04-04 11:41:39.863963
date : 2019-01-29 11:00:00 debut_traitement : 2019-04-04 11:42:46.032074
date : 2019-01-29 12:00:00 debut_traitement : 2019-04-04 11:43:47.519637
date : 2019-01-29 13:00:00 debut_traitement : 2019-04-04 11:44:49.935327


In [17]:
#MATRICE O-D !!II!
pd.pivot_table(test,values='l', index='origine', columns='destination',aggfunc='count', margins=True )

destination,A10,A62,A63,A630,A89,N10,All
origine,,,,,,,
A10,NaN,198.0,44.0,119.0,12.0,NaN,373
A62,57.0,NaN,2.0,57.0,7.0,127.0,250
A63,26.0,NaN,NaN,NaN,6.0,154.0,186
A630,NaN,65.0,NaN,NaN,87.0,828.0,980
A89,19.0,27.0,26.0,112.0,NaN,70.0,254
N10,NaN,195.0,504.0,329.0,47.0,NaN,1075
All,102.0,485.0,576.0,617.0,159.0,1179.0,3118


In [34]:
#df des immat en transit
#test.loc[(test['cameras'].apply(lambda x : x[0]==6)) & (pd.DatetimeIndex(test['date_cam_1']).hour<10) & (pd.DatetimeIndex(test['date_cam_1']).hour>=9)]
#retrouver ces immat dans la df des passages
test2=test.head(5).copy()
test2

,date_cam_1,immat,l,cameras,date_cam_2,tps_parcours,o_d,origine,destination
997,2019-01-29 06:46:03,83d4b2b69135eea6cd28c06f8cd90c15a49ad3db,1,"(10, 5)",2019-01-29 10:10:22,03:24:19,A62-N10,A62,N10
546,2019-01-29 06:26:21,b8cb362f853d39f5f0373e88f96f2cefa00ed823,1,"(10, 18, 13)",2019-01-29 08:10:04,01:43:43,A62-A63,A62,A63
1231,2019-01-29 06:39:37,318cedb332c7f9463eaad41cd8556bbd81da3ca6,1,"(8, 5)",2019-01-29 09:38:19,02:58:42,A89-N10,A89,N10
1375,2019-01-29 06:43:30,975c6ac836723cb5c6f18a90882af353b4ec9597,1,"(8, 5)",2019-01-29 08:45:42,02:02:12,A89-N10,A89,N10
1468,2019-01-29 06:46:26,fd48a76a4ddc3f1ce7a6030e6ee49c0a31414e01,1,"(8, 5)",2019-01-29 13:31:04,06:44:38,A89-N10,A89,N10


In [109]:
#df_journee.loc[df_journee['immat'].isin(test2['immat']) ]
#df_journee.loc[(df_journee['camera_id'].isin(test2['cameras'].any()))].camera_id.unique()
#faire une jointure avec le df de base pour ne conserver que les passages de transit du df
test3=test2.merge(df_journee, on='immat').sort_values(by='immat')
#filtrer selon les cameras présentent dans la liste et created est compris entre date_cam_1 et date_cam_2
test3.loc[(test3.apply(lambda x : x['camera_id'] in x['cameras'], axis=1)) & (test3.apply(lambda x : x['date_cam_1']<=x['created']<=x['date_cam_2'], axis=1))]
#filtrer si 

,cameras,camera_id
12,"(8, 5)",8
13,"(8, 5)",3
14,"(8, 5)",4
3,"(10, 18, 13)",10


In [127]:
test3.loc[(test3.apply(lambda x : x['camera_id'] in x['cameras'], axis=1)) & (test3.apply(lambda x : x['date_cam_1']<=x['created']<=x['date_cam_2'], axis=1))]

,date_cam_1,immat,l_x,cameras,date_cam_2,tps_parcours,o_d,origine,destination,camera_id,created,fiability,l_y,state
8,2019-01-29 06:39:37,318cedb332c7f9463eaad41cd8556bbd81da3ca6,1,"(8, 5)",2019-01-29 09:38:19,02:58:42,A89-N10,A89,N10,5,2019-01-29 09:38:19,99,1,PL
9,2019-01-29 06:39:37,318cedb332c7f9463eaad41cd8556bbd81da3ca6,1,"(8, 5)",2019-01-29 09:38:19,02:58:42,A89-N10,A89,N10,8,2019-01-29 06:39:37,88,0,PL
0,2019-01-29 06:46:03,83d4b2b69135eea6cd28c06f8cd90c15a49ad3db,1,"(10, 5)",2019-01-29 10:10:22,03:24:19,A62-N10,A62,N10,5,2019-01-29 10:10:22,99,1,BG
1,2019-01-29 06:46:03,83d4b2b69135eea6cd28c06f8cd90c15a49ad3db,1,"(10, 5)",2019-01-29 10:10:22,03:24:19,A62-N10,A62,N10,10,2019-01-29 06:46:03,59,0,BG
10,2019-01-29 06:43:30,975c6ac836723cb5c6f18a90882af353b4ec9597,1,"(8, 5)",2019-01-29 08:45:42,02:02:12,A89-N10,A89,N10,5,2019-01-29 08:45:42,99,1,FR
11,2019-01-29 06:43:30,975c6ac836723cb5c6f18a90882af353b4ec9597,1,"(8, 5)",2019-01-29 08:45:42,02:02:12,A89-N10,A89,N10,8,2019-01-29 06:43:30,99,0,FR
3,2019-01-29 06:26:21,b8cb362f853d39f5f0373e88f96f2cefa00ed823,1,"(10, 18, 13)",2019-01-29 08:10:04,01:43:43,A62-A63,A62,A63,10,2019-01-29 06:26:21,48,0,FR
4,2019-01-29 06:26:21,b8cb362f853d39f5f0373e88f96f2cefa00ed823,1,"(10, 18, 13)",2019-01-29 08:10:04,01:43:43,A62-A63,A62,A63,18,2019-01-29 07:26:43,57,0,FR
6,2019-01-29 06:26:21,b8cb362f853d39f5f0373e88f96f2cefa00ed823,1,"(10, 18, 13)",2019-01-29 08:10:04,01:43:43,A62-A63,A62,A63,13,2019-01-29 08:10:04,99,-1,FR
15,2019-01-29 06:46:26,fd48a76a4ddc3f1ce7a6030e6ee49c0a31414e01,1,"(8, 5)",2019-01-29 13:31:04,06:44:38,A89-N10,A89,N10,5,2019-01-29 13:31:04,99,1,BE


In [157]:
df_duree=df_journee.loc['2019-01-29 08:00:00':'2019-01-29 09:00:00']
df_duree_cam1=df_duree.loc[df_duree.loc[:,'camera_id']==12]
df_duree_autres_cam=df_journee.loc[(df_journee.loc[:,'immat'].isin(df_duree_cam1.loc[:,'immat']))]
df_autres_cam_temp_ok=df_duree_autres_cam.loc['2019-01-29 08:00:00':'2019-01-29 11:00:00']
groupe=(df_autres_cam_temp_ok.sort_index().reset_index().groupby('immat').agg({'camera_id':lambda x : tuple(x), 'l': lambda x : test_unicite_type(list(x),'1/2'),
                                                                        'created':lambda x: x.max()}))
df_agrege=df_duree_cam1.join(groupe,on='immat',lsuffix='_left')[['immat', 'l','camera_id','created']].rename(columns={'created':'date_cam_2', 'camera_id':'cameras'})
df_agrege=df_agrege.reset_index().rename(columns={'created':'date_cam_1'})
df_agrege['tps_parcours']=df_agrege.apply(lambda x : x.date_cam_2-x.date_cam_1, axis=1)
df_trajet=(df_agrege.loc[(df_agrege['cameras'].apply(lambda x : x[-1])==13)])
df_trajet

,date_cam_1,immat,l,cameras,date_cam_2,tps_parcours
0,2019-01-29 08:00:04,9642e28eac3b0552ed6e288261584dcb3845160f,-1,"(12, 3, 18, 13)",2019-01-29 10:14:52,02:14:48
3,2019-01-29 08:00:18,08fb4bbf9fb2099ab1d7c30409e77c6bb75b20b0,-1,"(12, 6, 3, 18, 13)",2019-01-29 10:27:14,02:26:56
75,2019-01-29 08:07:50,d6635bbacec4ec4a34f595f93322089b3defb5fd,-1,"(12, 6, 3, 18, 13)",2019-01-29 09:59:56,01:52:06
76,2019-01-29 08:07:55,a149e3e1c1236c9b4df23d8b4202725ccb2fef9d,-1,"(12, 6, 6, 3, 18, 13)",2019-01-29 10:08:14,02:00:19
97,2019-01-29 08:09:43,34be2912ad4c25051aa65cc54f717fa71d7eaa7e,-1,"(12, 6, 3, 18, 13)",2019-01-29 09:57:25,01:47:42
109,2019-01-29 08:10:17,1a7ad3e4b2da8036d18b3b54ce2799fd493fd16e,-1,"(12, 6, 3, 18, 13)",2019-01-29 09:52:18,01:42:01
186,2019-01-29 08:18:31,c650ea41ca7496b45df4aa8606b701488e85f2f2,-1,"(12, 6, 3, 18, 13)",2019-01-29 09:57:11,01:38:40
221,2019-01-29 08:20:13,7f9ee7b0f586ee0e4b3ca5bbf0fa505ca7f6a699,-1,"(12, 6, 3, 18, 13)",2019-01-29 09:57:41,01:37:28
316,2019-01-29 08:30:26,3a6d4ed727a9d5bdebcd51bc74563b1b12db3d34,-1,"(12, 6, 3, 18, 13)",2019-01-29 10:43:44,02:13:18
414,2019-01-29 08:38:51,6443465cf5f6b04f213bbf280e0874df27f12921,-1,"(12, 6, 3, 18, 13)",2019-01-29 10:10:40,01:31:49


In [171]:
test3=df_trajet.merge(df_duree_autres_cam.reset_index(), on='immat').sort_values(by='immat')
test3
test4=(test3.loc[(test3.apply(lambda x : x['camera_id'] in x['cameras'], axis=1)) 
                & (test3.apply(lambda x : x['date_cam_1']<=x['created']<=x['date_cam_2'], axis=1))].sort_values(['immat','created'])
       [['created','camera_id','immat','fiability','l_y','state']].rename(columns={'l_y':'l'}))
test4

,created,camera_id,immat,fiability,l,state
7,2019-01-29 08:00:18,12,08fb4bbf9fb2099ab1d7c30409e77c6bb75b20b0,99,1,FR
8,2019-01-29 08:14:16,6,08fb4bbf9fb2099ab1d7c30409e77c6bb75b20b0,89,1,FR
9,2019-01-29 08:40:32,3,08fb4bbf9fb2099ab1d7c30409e77c6bb75b20b0,99,0,FR
10,2019-01-29 09:05:52,18,08fb4bbf9fb2099ab1d7c30409e77c6bb75b20b0,89,1,FR
11,2019-01-29 10:27:14,13,08fb4bbf9fb2099ab1d7c30409e77c6bb75b20b0,69,-1,FR
31,2019-01-29 08:10:17,12,1a7ad3e4b2da8036d18b3b54ce2799fd493fd16e,99,0,ES
32,2019-01-29 08:24:29,6,1a7ad3e4b2da8036d18b3b54ce2799fd493fd16e,99,1,ES
33,2019-01-29 08:43:25,3,1a7ad3e4b2da8036d18b3b54ce2799fd493fd16e,99,1,ES
34,2019-01-29 09:05:45,18,1a7ad3e4b2da8036d18b3b54ce2799fd493fd16e,99,1,ES
35,2019-01-29 09:52:18,13,1a7ad3e4b2da8036d18b3b54ce2799fd493fd16e,99,-1,ES


In [141]:
def test_unicite_type(self,liste_l, mode='unique'):
        """test pour voir si un vehicule a ete toujours vu de la mme façon ou non
           en entre : liste de valeur de l iisues d'une df""" 
        if mode=='unique' : 
            if len(set(liste_l))==1 :
                return liste_l[0]
            else : 
                return -1
        elif mode=='1/2' :
            if any(liste_l)==1 : 
                return 1
            else : 
                return -1